## Welcome to Thermodynamics - Too Hot  to Handle 

In [ ]:
import numpy as np
from scipy.interpolate import griddata
import pandas as pd
import matplotlib.pyplot as plt
from apdx_functions import *

In [ ]:
from Rankine_solver import *

vars = define_empty_variables()

# give the solver *one* piece of information per state
vars['2']['T'] = -8  # K
vars['3']['P'] = 0.8 # MPa
vars['m_dot'] = 0.1 # kg/s

solved = solve_r_rankine_cycle(vars)

print(solved['Wc_dot'], solved['Qh_dot'])
solved


In [ ]:
# Load the new uploaded CSV
file_path_new = 'Data/9c-Superheated-R134a.csv'
df_new = pd.read_csv(file_path_new, header=1)


# Convert Pressure and Temperature columns to numeric
df_new['P'] = pd.to_numeric(df_new['P'], errors='coerce')
df_new['T'] = pd.to_numeric(df_new['T'], errors='coerce')

# Now plot
plt.figure(figsize=(10, 6))
plt.scatter(df_new['P'], df_new['T'], s=10)
plt.title('2D Grid: Pressure vs Temperature (Superheated R134a)')
plt.xlabel('Pressure (MPa)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()

In [ ]:
# Define the grid over which we want to interpolate
pressure_vals = np.linspace(df_new['P'].min(), df_new['P'].max(), 200)
temperature_vals = np.linspace(df_new['T'].min(), df_new['T'].max(), 200)

# Create a grid
P_grid, T_grid = np.meshgrid(pressure_vals, temperature_vals)

# Pick a property to interpolate, for example, entropy (s)
# First, ensure entropy is numeric
df_new['s'] = pd.to_numeric(df_new['s'], errors='coerce')

# Prepare points and values
points = np.column_stack((df_new['P'], df_new['T']))
values = df_new['s']

print(griddata(points, values, (0.8, 100), method='linear'))

# Interpolate
S_grid = griddata(points, values, (P_grid, T_grid), method='linear')

# Plot the interpolated surface
plt.figure(figsize=(10, 6))
contour = plt.contourf(P_grid, T_grid, S_grid, levels=50)
plt.colorbar(contour, label='Entropy s (kJ/kg·K)')
plt.scatter(df_new['P'], df_new['T'], s=10)
plt.title('Interpolated Entropy (s) over Pressure-Temperature Grid')
plt.xlabel('Pressure (MPa)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()

In [ ]:
from scipy.interpolate import LinearNDInterpolator, griddata
import numpy as np
import pandas as pd

# Load the new uploaded CSV
file_path_new = 'Data/9c-Superheated-R134a.csv'
df_new = pd.read_csv(file_path_new, header=1)


# Convert Pressure and Temperature columns to numeric
df_new['P'] = pd.to_numeric(df_new['P'], errors='coerce')
df_new['T'] = pd.to_numeric(df_new['T'], errors='coerce')
df_new['s'] = pd.to_numeric(df_new['s'], errors='coerce')

z, zname = df_new['P'].values, 'Pressure (MPa)'
x, xname = df_new['T'].values, 'Temperature (°C)'
y, yname = df_new['s'].values, 'Entropy (s)'

xy = np.c_[x, y]   # or just list(zip(x, y))
lut2 = LinearNDInterpolator(xy, z)
X = np.linspace(min(x), max(x))
Y = np.linspace(min(y), max(y))
X, Y = np.meshgrid(X, Y)

# Prepare points and values
points = np.column_stack((df_new['P'], df_new['T']))
property = 's'

print(griddata(points, df_new[property], (0.8, 100), method='linear'))


import plotly.graph_objects as go

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(z=lut2(X, Y), x=X, y=Y, colorscale='Viridis')])

# Add scatter points
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=3, color='black'), name='Data Points'))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title=xname,
    yaxis_title=yname,
    zaxis_title=zname
), title='3D Surface Plot with Data Points')

# Show the plot
fig.show()